# Basic Data

In [ ]:
N_PHONEMES = 41
PHONEME_LIST = [
    " ",
    "SIL",
    "SPN",
    "AA",
    "AE",
    "AH",
    "AO",
    "AW",
    "AY",
    "B",
    "CH",
    "D",
    "DH",
    "EH",
    "ER",
    "EY",
    "F",
    "G",
    "H",
    "IH",
    "IY",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "OW",
    "OY",
    "P",
    "R",
    "S",
    "SH",
    "T",
    "TH",
    "UH",
    "UW",
    "V",
    "W",
    "Y",
    "Z",
    "ZH"
]

PHONEME_MAP = [
    " ",
    ".", #SIL
    "!", #SPN
    "a", #AA
    "A", #AE
    "h", #AH
    "o", #AO
    "w", #AW
    "y", #AY
    "b", #B
    "c", #CH
    "d", #D
    "D", #DH
    "e", #EH
    "r", #ER
    "E", #EY
    "f", #F
    "g", #G
    "H", #H
    "i", #IH 
    "I", #IY
    "j", #JH
    "k", #K
    "l", #L
    "m", #M
    "n", #N
    "N", #NG
    "O", #OW
    "Y", #OY
    "p", #P 
    "R", #R
    "s", #S
    "S", #SH
    "t", #T
    "T", #TH
    "u", #UH
    "U", #UW
    "v", #V
    "W", #W
    "?", #Y
    "z", #Z
    "Z" #ZH
]

assert len(PHONEME_LIST) == len(PHONEME_MAP)
assert len(set(PHONEME_MAP)) == len(PHONEME_MAP)



# Preparation

In [ ]:
!nvidia-smi
#torch.cuda.empty_cache()

Sun Apr 11 18:08:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence,pack_padded_sequence,pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

from PIL import Image

import torchvision   
import matplotlib.pyplot as plt
import os, zipfile, tarfile

!pip install --upgrade --force-reinstall --no-deps kaggle
!apt install unzip


!pip install python-Levenshtein
from Levenshtein import distance

!pip install wget
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .
from ctcdecode import CTCBeamDecoder



     |████████████████████████████████| 61kB 5.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=88273ebe1914e88d22bdecd283bd5cbb2c88056954b1650f2e03a962cd476b63
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149824 sha256=dfeaed41b207de95f2821de9a4f5bf74fcda8e3bd5b3faf4f5bc512fcd926331
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/MyDrive/11785hw3p2
%pwd
%ls

# Data Loading

In [ ]:
print("loading training labels")
training_labels = [torch.from_numpy(np.array(x)).int() for x in np.load('train_labels.npy',allow_pickle=True)]

print("loading training data")
training_data = [torch.from_numpy(x).float() for x in np.load('train.npy',allow_pickle=True)]




print("loading dev labels")
dev_labels = [torch.from_numpy(np.array(x)).int() for x in np.load('dev_labels.npy',allow_pickle=True)]

print("loading dev data")
dev_data = [torch.from_numpy(x).float() for x in np.load('dev.npy',allow_pickle=True)]



loading training labels
loading training data
loading dev labels
loading dev data


In [ ]:
# Dataset definition

class SpeechDataset(Dataset):
    
    # load the dataset
    def __init__(self, x, y):
       
        self.x = x
        self.y = y 

        assert(len(x) == len(y))
        self.length = len(x)
        
    # number of rows in the dataset
    def __len__(self):
        return self.length
 
    # get a row at an index
    def __getitem__(self, index):
        return self.x[index],self.y[index]
    
    def collate_fn(batch):
        
        batch_x = [x for x,_ in batch]
        batch_y = [y for _,y in batch]

        length_x = torch.Tensor([x.shape[0] for x,_ in batch]).int()
        length_y = torch.Tensor([y.shape[0] for _,y in batch]).int()
        
        ### Convert batched data and labels to tensors (2 lines)
        batch_x = pad_sequence(batch_x)
        batch_y = pad_sequence(batch_y, batch_first=True)
        
        ### Return batched data and labels (1 line)
        return batch_x, batch_y, length_x, length_y




In [ ]:

# Training dataloader
train_data = SpeechDataset(training_data, training_labels)
train_args = dict(shuffle = True, batch_size = 16,drop_last=True,collate_fn=SpeechDataset.collate_fn)
train_loader = DataLoader(train_data, **train_args)


# Validation dataloader
val_data = SpeechDataset(dev_data, dev_labels)
val_args = dict(shuffle = False, batch_size = 16,drop_last=False,collate_fn=SpeechDataset.collate_fn)
val_loader = DataLoader(val_data, **val_args)






# Model

In [ ]:

class ConvBlock(nn.Module):
  def __init__(self,in_channel,out_channel=None):
    super(ConvBlock, self).__init__()
    if out_channel is None:
      out_channel = in_channel
    self.layers = nn.Sequential(
      nn.Conv1d(in_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False),
      nn.BatchNorm1d(out_channel),
      nn.ReLU(),
      nn.AvgPool1d(3, stride=1, padding=1, count_include_pad=True)
    )

  def forward(self, x):
    out = self.layers(x)
    return out

# Convolution - BILSTM
class Model(nn.Module):
    def __init__(self, in_size, out_vocab, hidden_size):
        super(Model, self).__init__()

        

        self.layers = nn.Sequential(
            ConvBlock(in_size,out_channel=hidden_size),
            nn.Dropout(p=0.5)
        )

        self.lstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True,num_layers=2,dropout=0.4)

        self.linear = nn.Linear(hidden_size*2, out_vocab)
    
    def forward(self, X, lengths):
        X = X.transpose(0,1).transpose(1,2)
        X = self.layers(X)
        X = X.transpose(1,2).transpose(0,1)

        packed_X = pack_padded_sequence(X, lengths,enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        
        out, out_lens = pad_packed_sequence(packed_out)

        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.linear(out).log_softmax(2)
        
        return out, out_lens

# Training

In [ ]:
in_features = 40
out_features = 42   # include space


hidden_size = 128
numEpochs = 500 
learningRate = 2e-3
weightDecay = 5e-6
beam_width = 10



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Model(in_features, out_features,hidden_size)
model = model.to(device)

criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=1,verbose=True)

decoder = CTCBeamDecoder(PHONEME_MAP, beam_width=beam_width, num_processes=os.cpu_count(), log_probs_input=True)




In [ ]:
def check_levenshtein(out,out_length,target,target_length):
  out, _, _, out_length = decoder.decode(torch.transpose(out,0,1),out_length)
  total_distance = 0
  for i in range(len(out)):
    best_seq = out[i, 0, :out_length[i, 0]]
    best_pron = ''.join(PHONEME_MAP[j] for j in best_seq)

    target_seq = target[i, :target_length[i]]
    target_pron = ''.join(PHONEME_MAP[j] for j in target_seq)
    #print('best_pron',best_pron)
    #print('target_pron',target_pron)

  
    dist = distance(best_pron, target_pron)
    #print('distance',dist)
    total_distance += dist

  return total_distance

In [ ]:
def train_model(train_loader,model):
  model.train()

  total_loss = 0
  total_lev = 0

  for batch, (input,target, input_length, target_length) in tqdm(enumerate(train_loader),total=len(train_loader)):
    optimizer.zero_grad()


    input,target = input.to(device),target.to(device)
    out, out_lens = model(input,input_length)
    
    loss = criterion(out, target, out_lens, target_length)
    
    total_loss += loss.item()
    
    loss.backward()
    optimizer.step()

    
    # speed up
    #lev_dist = check_levenshtein(out,out_lens,target,target_length)
    #total_lev += lev_dist
    total_lev = 0

    if batch % 10 == 0:
      print('Batch: {}\tAvg-Loss: {:.4f}\tAvg-Lev-Dist: {:.2f}'.format(batch+1, total_loss/(batch+1),total_lev/(batch+1)))


    torch.cuda.empty_cache()
    del out
    del out_lens
    del loss

  return total_loss,total_lev

def validate_model(val_loader,model):
  model.eval()

  total_loss = 0
  total_lev = 0
  batch_total = 0

  for batch, (input,target, input_length, target_length) in tqdm(enumerate(val_loader),total=len(val_loader)):
    optimizer.zero_grad()
    input,target = input.to(device),target.to(device)

    out, out_lens = model(input,input_length)
    loss = criterion(out, target, out_lens, target_length)
    total_loss += loss.item()
    

    lev_dist = check_levenshtein(out,out_lens,target,target_length)
    total_lev += lev_dist
    if batch % 10 == 0:
      print('Batch: {}\tAvg-Loss: {:.4f}\tAvg-Lev-Dist: {:.2f}'.format(batch+1, total_loss/(batch+1),total_lev/(batch+1)))


    torch.cuda.empty_cache()
    del out
    del out_lens
    del loss

    batch_total += 1

  return total_loss/batch_total,total_lev/batch_total

In [ ]:
#temp = torch.load(f"/content/gdrive/MyDrive/11785hw3p2/CLDNN-thick-LSTM_epoch="+str(29)+"_early-model.pt")
#model.load_state_dict(temp['model_state_dict'])
#optimizer.load_state_dict(temp['optimizer_state_dict'])

#val_loss, val_dist = validate_model(val_loader,model)
#print('Epoch Result:{}\tVal-Loss: {:.4f}\tVal-Lev-Dist: {:.2f}'.format(0, val_loss,val_dist))



In [ ]:

for epoch in range(numEpochs):
    print(f"Epoch {epoch}")

    training_loss, training_dist = train_model(train_loader,model)

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    },f"/content/gdrive/MyDrive/11785hw3p2/conv_1d-1-layer-lstm_epoch="+str(epoch)+".pt")

    val_loss, val_dist = validate_model(val_loader,model)

    print('Epoch Result:{}\tVal-Loss: {:.4f}\tVal-Lev-Dist: {:.2f}'.format(epoch, val_loss,val_dist))
    
    scheduler.step(val_loss)



Epoch 0


Batch: 1	Avg-Loss: 30.5403	Avg-Lev-Dist: 0.00
Batch: 11	Avg-Loss: 11.7969	Avg-Lev-Dist: 0.00
Batch: 21	Avg-Loss: 7.9736	Avg-Lev-Dist: 0.00
Batch: 31	Avg-Loss: 6.5384	Avg-Lev-Dist: 0.00
Batch: 41	Avg-Loss: 5.7882	Avg-Lev-Dist: 0.00
Batch: 51	Avg-Loss: 5.3273	Avg-Lev-Dist: 0.00
Batch: 61	Avg-Loss: 5.0143	Avg-Lev-Dist: 0.00
Batch: 71	Avg-Loss: 4.7900	Avg-Lev-Dist: 0.00
Batch: 81	Avg-Loss: 4.6211	Avg-Lev-Dist: 0.00
Batch: 91	Avg-Loss: 4.4890	Avg-Lev-Dist: 0.00
Batch: 101	Avg-Loss: 4.3836	Avg-Lev-Dist: 0.00
Batch: 111	Avg-Loss: 4.2969	Avg-Lev-Dist: 0.00
Batch: 121	Avg-Loss: 4.2251	Avg-Lev-Dist: 0.00
Batch: 131	Avg-Loss: 4.1635	Avg-Lev-Dist: 0.00
Batch: 141	Avg-Loss: 4.1102	Avg-Lev-Dist: 0.00
Batch: 151	Avg-Loss: 4.0645	Avg-Lev-Dist: 0.00
Batch: 161	Avg-Loss: 4.0245	Avg-Lev-Dist: 0.00
Batch: 171	Avg-Loss: 3.9885	Avg-Lev-Dist: 0.00
Batch: 181	Avg-Loss: 3.9575	Avg-Lev-Dist: 0.00
Batch: 191	Avg-Loss: 3.9292	Avg-Lev-Dist: 0.00
Batch: 201	Avg-Loss: 3.9034	Avg-Lev-Dist: 0.00
Batch: 211	Avg-Loss: 3

Batch: 1	Avg-Loss: 2.4279	Avg-Lev-Dist: 1192.00
Batch: 11	Avg-Loss: 2.3604	Avg-Lev-Dist: 967.00
Batch: 21	Avg-Loss: 2.3563	Avg-Lev-Dist: 1013.14
Batch: 31	Avg-Loss: 2.3608	Avg-Lev-Dist: 1028.42
Batch: 41	Avg-Loss: 2.3558	Avg-Lev-Dist: 1008.93
Batch: 51	Avg-Loss: 2.3561	Avg-Lev-Dist: 994.04
Batch: 61	Avg-Loss: 2.3571	Avg-Lev-Dist: 984.34
Batch: 71	Avg-Loss: 2.3608	Avg-Lev-Dist: 975.35
Batch: 81	Avg-Loss: 2.3645	Avg-Lev-Dist: 974.62
Batch: 91	Avg-Loss: 2.3645	Avg-Lev-Dist: 963.02
Batch: 101	Avg-Loss: 2.3658	Avg-Lev-Dist: 971.45
Batch: 111	Avg-Loss: 2.3686	Avg-Lev-Dist: 982.48
Batch: 121	Avg-Loss: 2.3698	Avg-Lev-Dist: 981.74
Batch: 131	Avg-Loss: 2.3720	Avg-Lev-Dist: 986.86

Epoch Result:0	Val-Loss: 2.3725	Val-Lev-Dist: 980.19
Epoch 1


Batch: 1	Avg-Loss: 2.4415	Avg-Lev-Dist: 0.00
Batch: 11	Avg-Loss: 2.3780	Avg-Lev-Dist: 0.00
Batch: 21	Avg-Loss: 2.3711	Avg-Lev-Dist: 0.00
Batch: 31	Avg-Loss: 2.3561	Avg-Lev-Dist: 0.00
Batch: 41	Avg-Loss: 2.3420	Avg-Lev-Dist: 0.00
Batch: 51	Avg-Loss: 2.3340	Avg-Lev-Dist: 0.00
Batch: 61	Avg-Loss: 2.3213	Avg-Lev-Dist: 0.00
Batch: 71	Avg-Loss: 2.3128	Avg-Lev-Dist: 0.00
Batch: 81	Avg-Loss: 2.3021	Avg-Lev-Dist: 0.00
Batch: 91	Avg-Loss: 2.2925	Avg-Lev-Dist: 0.00
Batch: 101	Avg-Loss: 2.2787	Avg-Lev-Dist: 0.00
Batch: 111	Avg-Loss: 2.2681	Avg-Lev-Dist: 0.00
Batch: 121	Avg-Loss: 2.2569	Avg-Lev-Dist: 0.00
Batch: 131	Avg-Loss: 2.2487	Avg-Lev-Dist: 0.00
Batch: 141	Avg-Loss: 2.2429	Avg-Lev-Dist: 0.00
Batch: 151	Avg-Loss: 2.2351	Avg-Lev-Dist: 0.00
Batch: 161	Avg-Loss: 2.2246	Avg-Lev-Dist: 0.00
Batch: 171	Avg-Loss: 2.2150	Avg-Lev-Dist: 0.00
Batch: 181	Avg-Loss: 2.2060	Avg-Lev-Dist: 0.00
Batch: 191	Avg-Loss: 2.1954	Avg-Lev-Dist: 0.00
Batch: 201	Avg-Loss: 2.1854	Avg-Lev-Dist: 0.00
Batch: 211	Avg-Loss: 2.1

Batch: 1	Avg-Loss: 1.3499	Avg-Lev-Dist: 567.00
Batch: 11	Avg-Loss: 1.3035	Avg-Lev-Dist: 441.64
Batch: 21	Avg-Loss: 1.3146	Avg-Lev-Dist: 465.29
Batch: 31	Avg-Loss: 1.3257	Avg-Lev-Dist: 471.84
Batch: 41	Avg-Loss: 1.3160	Avg-Lev-Dist: 464.07
Batch: 51	Avg-Loss: 1.3213	Avg-Lev-Dist: 458.00
Batch: 61	Avg-Loss: 1.3196	Avg-Lev-Dist: 454.70
Batch: 71	Avg-Loss: 1.3243	Avg-Lev-Dist: 451.14
Batch: 81	Avg-Loss: 1.3322	Avg-Lev-Dist: 451.73
Batch: 91	Avg-Loss: 1.3323	Avg-Lev-Dist: 447.03
Batch: 101	Avg-Loss: 1.3323	Avg-Lev-Dist: 451.40
Batch: 111	Avg-Loss: 1.3333	Avg-Lev-Dist: 456.73
Batch: 121	Avg-Loss: 1.3351	Avg-Lev-Dist: 456.90
Batch: 131	Avg-Loss: 1.3345	Avg-Lev-Dist: 458.66

Epoch Result:1	Val-Loss: 1.3363	Val-Lev-Dist: 456.77
Epoch 2


...

Batch: 1	Avg-Loss: 0.3655	Avg-Lev-Dist: 0.00
Batch: 11	Avg-Loss: 0.4053	Avg-Lev-Dist: 0.00
Batch: 21	Avg-Loss: 0.4165	Avg-Lev-Dist: 0.00
Batch: 31	Avg-Loss: 0.4212	Avg-Lev-Dist: 0.00
Batch: 41	Avg-Loss: 0.4209	Avg-Lev-Dist: 0.00
Batch: 51	Avg-Loss: 0.4218	Avg-Lev-Dist: 0.00
Batch: 61	Avg-Loss: 0.4239	Avg-Lev-Dist: 0.00
Batch: 71	Avg-Loss: 0.4195	Avg-Lev-Dist: 0.00
Batch: 81	Avg-Loss: 0.4183	Avg-Lev-Dist: 0.00
Batch: 91	Avg-Loss: 0.4181	Avg-Lev-Dist: 0.00
Batch: 101	Avg-Loss: 0.4160	Avg-Lev-Dist: 0.00
Batch: 111	Avg-Loss: 0.4181	Avg-Lev-Dist: 0.00
Batch: 121	Avg-Loss: 0.4169	Avg-Lev-Dist: 0.00
Batch: 131	Avg-Loss: 0.4151	Avg-Lev-Dist: 0.00
Batch: 141	Avg-Loss: 0.4155	Avg-Lev-Dist: 0.00
Batch: 151	Avg-Loss: 0.4163	Avg-Lev-Dist: 0.00
Batch: 161	Avg-Loss: 0.4184	Avg-Lev-Dist: 0.00
Batch: 171	Avg-Loss: 0.4186	Avg-Lev-Dist: 0.00
Batch: 181	Avg-Loss: 0.4178	Avg-Lev-Dist: 0.00
Batch: 191	Avg-Loss: 0.4166	Avg-Lev-Dist: 0.00
Batch: 201	Avg-Loss: 0.4166	Avg-Lev-Dist: 0.00
Batch: 211	Avg-Loss: 0.4

Batch: 1	Avg-Loss: 0.4971	Avg-Lev-Dist: 203.00
Batch: 11	Avg-Loss: 0.4894	Avg-Lev-Dist: 160.09
Batch: 21	Avg-Loss: 0.4821	Avg-Lev-Dist: 165.81
Batch: 31	Avg-Loss: 0.5022	Avg-Lev-Dist: 168.55
Batch: 41	Avg-Loss: 0.4929	Avg-Lev-Dist: 165.12
Batch: 51	Avg-Loss: 0.4973	Avg-Lev-Dist: 164.80
Batch: 61	Avg-Loss: 0.4889	Avg-Lev-Dist: 162.57
Batch: 71	Avg-Loss: 0.4911	Avg-Lev-Dist: 161.76
Batch: 81	Avg-Loss: 0.4945	Avg-Lev-Dist: 162.44
Batch: 91	Avg-Loss: 0.4934	Avg-Lev-Dist: 161.57
Batch: 101	Avg-Loss: 0.4938	Avg-Lev-Dist: 163.83
Batch: 111	Avg-Loss: 0.4951	Avg-Lev-Dist: 166.18
Batch: 121	Avg-Loss: 0.4986	Avg-Lev-Dist: 166.75
Batch: 131	Avg-Loss: 0.4986	Avg-Lev-Dist: 167.52

Epoch Result:98	Val-Loss: 0.5004	Val-Lev-Dist: 167.09
Epoch 99


Batch: 1	Avg-Loss: 0.4239	Avg-Lev-Dist: 0.00
Batch: 11	Avg-Loss: 0.4210	Avg-Lev-Dist: 0.00
Batch: 21	Avg-Loss: 0.4182	Avg-Lev-Dist: 0.00
Batch: 31	Avg-Loss: 0.4256	Avg-Lev-Dist: 0.00
Batch: 41	Avg-Loss: 0.4234	Avg-Lev-Dist: 0.00
Batch: 51	Avg-Loss: 0.4222	Avg-Lev-Dist: 0.00
Batch: 61	Avg-Loss: 0.4275	Avg-Lev-Dist: 0.00
Batch: 71	Avg-Loss: 0.4283	Avg-Lev-Dist: 0.00
Batch: 81	Avg-Loss: 0.4294	Avg-Lev-Dist: 0.00
Batch: 91	Avg-Loss: 0.4270	Avg-Lev-Dist: 0.00
Batch: 101	Avg-Loss: 0.4265	Avg-Lev-Dist: 0.00
Batch: 111	Avg-Loss: 0.4240	Avg-Lev-Dist: 0.00
Batch: 121	Avg-Loss: 0.4236	Avg-Lev-Dist: 0.00
Batch: 131	Avg-Loss: 0.4211	Avg-Lev-Dist: 0.00
Batch: 141	Avg-Loss: 0.4223	Avg-Lev-Dist: 0.00
Batch: 151	Avg-Loss: 0.4218	Avg-Lev-Dist: 0.00
Batch: 161	Avg-Loss: 0.4202	Avg-Lev-Dist: 0.00
Batch: 171	Avg-Loss: 0.4208	Avg-Lev-Dist: 0.00
Batch: 181	Avg-Loss: 0.4214	Avg-Lev-Dist: 0.00
Batch: 191	Avg-Loss: 0.4209	Avg-Lev-Dist: 0.00
Batch: 201	Avg-Loss: 0.4215	Avg-Lev-Dist: 0.00
Batch: 211	Avg-Loss: 0.4

Batch: 1	Avg-Loss: 0.5001	Avg-Lev-Dist: 204.00
Batch: 11	Avg-Loss: 0.4886	Avg-Lev-Dist: 159.82
Batch: 21	Avg-Loss: 0.4818	Avg-Lev-Dist: 166.00
Batch: 31	Avg-Loss: 0.5017	Avg-Lev-Dist: 168.81
Batch: 41	Avg-Loss: 0.4924	Avg-Lev-Dist: 165.37
Batch: 51	Avg-Loss: 0.4967	Avg-Lev-Dist: 164.69
Batch: 61	Avg-Loss: 0.4884	Avg-Lev-Dist: 162.34
Batch: 71	Avg-Loss: 0.4905	Avg-Lev-Dist: 161.44
Batch: 81	Avg-Loss: 0.4940	Avg-Lev-Dist: 162.10
Batch: 91	Avg-Loss: 0.4930	Avg-Lev-Dist: 161.27
Batch: 101	Avg-Loss: 0.4935	Avg-Lev-Dist: 163.47
Batch: 111	Avg-Loss: 0.4948	Avg-Lev-Dist: 165.79
Batch: 121	Avg-Loss: 0.4984	Avg-Lev-Dist: 166.47
Batch: 131	Avg-Loss: 0.4982	Avg-Lev-Dist: 167.28

Epoch Result:99	Val-Loss: 0.5000	Val-Lev-Dist: 166.86
Epoch 100


Batch: 1	Avg-Loss: 0.4120	Avg-Lev-Dist: 0.00
Batch: 11	Avg-Loss: 0.4269	Avg-Lev-Dist: 0.00
Batch: 21	Avg-Loss: 0.4193	Avg-Lev-Dist: 0.00
Batch: 31	Avg-Loss: 0.4104	Avg-Lev-Dist: 0.00
Batch: 41	Avg-Loss: 0.4059	Avg-Lev-Dist: 0.00
Batch: 51	Avg-Loss: 0.4136	Avg-Lev-Dist: 0.00
Batch: 61	Avg-Loss: 0.4086	Avg-Lev-Dist: 0.00
Batch: 71	Avg-Loss: 0.4164	Avg-Lev-Dist: 0.00
Batch: 81	Avg-Loss: 0.4140	Avg-Lev-Dist: 0.00
Batch: 91	Avg-Loss: 0.4164	Avg-Lev-Dist: 0.00
Batch: 101	Avg-Loss: 0.4158	Avg-Lev-Dist: 0.00
Batch: 111	Avg-Loss: 0.4167	Avg-Lev-Dist: 0.00
Batch: 121	Avg-Loss: 0.4152	Avg-Lev-Dist: 0.00
Batch: 131	Avg-Loss: 0.4159	Avg-Lev-Dist: 0.00
Batch: 141	Avg-Loss: 0.4191	Avg-Lev-Dist: 0.00
Batch: 151	Avg-Loss: 0.4189	Avg-Lev-Dist: 0.00
Batch: 161	Avg-Loss: 0.4181	Avg-Lev-Dist: 0.00
Batch: 171	Avg-Loss: 0.4188	Avg-Lev-Dist: 0.00
Batch: 181	Avg-Loss: 0.4181	Avg-Lev-Dist: 0.00
Batch: 191	Avg-Loss: 0.4199	Avg-Lev-Dist: 0.00
Batch: 201	Avg-Loss: 0.4213	Avg-Lev-Dist: 0.00
Batch: 211	Avg-Loss: 0.4

# Generate Test Result


In [ ]:
# Test Results

print("loading test data")
test_data = [torch.from_numpy(x).float() for x in np.load('test.npy',allow_pickle=True)]

In [ ]:

# Dataset definition

class SpeechDatasetTest(Dataset):
    
    # load the dataset
    def __init__(self, x):
       
        self.x = x

        self.length = len(x)
        
    # number of rows in the dataset
    def __len__(self):
        return self.length
 
    # get a row at an index
    def __getitem__(self, index):
        return self.x[index]
    
    def collate_fn(batch):
        
        batch_x = [x for x in batch]

        length_x = torch.Tensor([x.shape[0] for x in batch]).int()
        
        ### Convert batched data and labels to tensors (2 lines)
        batch_x = pad_sequence(batch_x)
        
        ### Return batched data and labels (1 line)
        return batch_x, length_x

In [ ]:

# Test Data
test_data = SpeechDatasetTest(test_data)
test_args = dict(shuffle = False, batch_size = 16,drop_last=False,collate_fn=SpeechDatasetTest.collate_fn)
test_loader = DataLoader(test_data, **test_args)

In [ ]:
def get_seq(out,out_length):
  out, _, _, out_length = decoder.decode(torch.transpose(out,0,1),out_length)
  result = []
  for i in range(len(out)):
    best_seq = out[i, 0, :out_length[i, 0]]
    best_pron = ''.join(PHONEME_MAP[j] for j in best_seq)
    result.append(best_pron)
    

  return result

In [ ]:
def test_model(test_loader,model):
  model.eval()

  result = []

  for batch, (input,input_length) in tqdm(enumerate(test_loader),total=len(test_loader)):
    optimizer.zero_grad()
    input = input.to(device)

    out, out_length = model(input,input_length)
    
    out_seq = get_seq(out,out_length)
    result += out_seq

    torch.cuda.empty_cache()
    del out
    del out_length

  return result

In [ ]:
# Actual Test Result
result = test_model(test_loader,model)
print("Result shape:",len(result))
print("First seq:",result[0])
test_length = len(result)

# Save Result

In [ ]:
import csv
df = pd.DataFrame({'id': [i for i in range(test_length)], 'label': result})
print(df.shape)
df.to_csv('/content/gdrive/MyDrive/11785hw3p2/test_results_1.csv',index=False,quoting=csv.QUOTE_NONE,escapechar=" ",)

# Debugger for Levenshtein

In [ ]:
!nvidia-smi
torch.cuda.empty_cache()

In [ ]:
for epoch in range(50):
    print(f"Epoch {epoch}")

    model.train()

    total_loss = 0
    total_lev = 0

    for batch, (input,target, input_length, target_length) in tqdm(enumerate(train_loader)):
      optimizer.zero_grad()


      input,target = input.to(device),target.to(device)
      out, out_lens = model(input,input_length)
      #print("Arg to criterion loss",out,target,out_lens,target_length)
      print("Shapes input to loss",out.shape, target.shape, out_lens.shape, target_length.shape)
      loss = criterion(out, target, out_lens, target_length)
      print("loss calculation complete")
      total_loss += loss.item()
      
      loss.backward()
      optimizer.step()

      print("stepped loss")

      # LEVENSHTEIN -----------------------------------------------

      out_length = out_lens
      #print("Out",out)
      #print("Target",target)
      
      #out = torch.transpose(out,0,1)
      print("Out length",out_length)
      #print("Out shape levenshtein pre-decode:",out.shape,out_length.shape)
      out, _, _, out_length = decoder.decode(torch.transpose(out,0,1),out_length)
      #print("Out shape levenshtein post-decode",out.shape,out_length.shape)
      total_distance = 0
      for i in range(len(out)):
        best_seq = out[i, 0, :out_length[i, 0]]
        print(f"best_seq-[{i}]",best_seq)
        best_pron = ''.join(PHONEME_MAP[j] for j in best_seq)
        print(f"best_pron-[{i}]",best_pron)

        target_seq = target[i, :target_length[i]]
        print(f"target_seq-[{i}]",target_seq)
        target_pron = ''.join(PHONEME_MAP[j] for j in target_seq)
        print(f"target_pron-[{i}]",target_pron)

      
        total_distance += distance(best_pron, target_pron)
      #total_distance = 0
      # ----------------------------------------------------------------------------
      lev_dist = total_distance #check_levenshtein(out,out_lens,target,target_length)
      total_lev += lev_dist

      print('Batch: {}\tAvg-Loss: {:.4f}\tAvg-Lev-Dist: {:.2f}'.format(batch+1, total_loss/(batch+1),total_lev/(batch+1)))


      torch.cuda.empty_cache()
      del out
      del out_lens
      del loss

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    },f"/content/gdrive/MyDrive/11785hw3p2/bilstm_epoch="+str(epoch)+"_early-model.pt")

    val_loss, val_dist = validate_model(val_loader,model)

    scheduler.step(val_loss)

    print('Epoch Result:{}\tVal-Loss: {:.4f}\tVal-Lev-Dist: {:.2f}'.format(epoch, val_loss,val_dist))



In [ ]:
print(target_length[0])
print(target.shape)

In [ ]:
print("Phoneme map length",len(PHONEME_MAP))
print("Phoneme list length",len(PHONEME_LIST))
#CTCBeamDecoder(PHONEME_MAP, beam_width=beam_width, num_processes=os.cpu_count(), log_probs_input=True)

# Old Models

In [ ]:

# Traditional LSTM for 'bilstm_epoch=28_early-model.pt'
'''class Model(nn.Module):
    def __init__(self, in_size, out_vocab, hidden_size):
        super(Model, self).__init__()
        self.lstm1 = nn.LSTM(in_size, hidden_size, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_size*2, hidden_size, bidirectional=True)
        self.output = nn.Linear(hidden_size * 2, out_vocab)
    
    def forward(self, X, lengths):
        packed_X = pack_padded_sequence(X, lengths,enforce_sorted=False)
        packed_out = self.lstm1(packed_X)[0]
        packed_out = self.lstm2(packed_out)[0]
        out, out_lens = pad_packed_sequence(packed_out)
        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.output(out).log_softmax(2)
        return out, out_lens'''

In [ ]:

# Old model for "/content/gdrive/MyDrive/11785hw3p2/conv-bilstm_epoch="+str(epoch)+"_early-model.pt"
class ConvBlock(nn.Module):
  def __init__(self,in_channel,out_channel=None):
    super(ConvBlock, self).__init__()
    if out_channel is None:
      out_channel = in_channel
    self.layers = nn.Sequential(
      nn.Conv1d(in_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False),
      nn.BatchNorm1d(out_channel),
      nn.ReLU(),
      nn.AvgPool1d(3, stride=1, padding=1, count_include_pad=True)
    )

  def forward(self, x):
    out = self.layers(x)
    return out

# Convolution - BILSTM
class Model(nn.Module):
    def __init__(self, in_size, out_vocab, hidden_size):
        super(Model, self).__init__()

        

        self.layers = nn.Sequential(
            ConvBlock(in_size,out_channel=hidden_size),
            ConvBlock(hidden_size,out_channel=hidden_size*2),
        )

        self.lstm1 = nn.LSTM(hidden_size*2, hidden_size, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_size*2, hidden_size, bidirectional=True)

        self.linear = nn.Linear(hidden_size*2, out_vocab)
    
    def forward(self, X, lengths):
        X = X.transpose(0,1).transpose(1,2)
        X = self.layers(X)
        X = X.transpose(1,2).transpose(0,1)

        packed_X = pack_padded_sequence(X, lengths,enforce_sorted=False)
        packed_out = self.lstm1(packed_X)[0]
        packed_out = self.lstm2(packed_out)[0]
        
        out, out_lens = pad_packed_sequence(packed_out)

        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.linear(out).log_softmax(2)
        
        return out, out_lens

In [ ]:
# For old model "/content/gdrive/MyDrive/11785hw3p2/cldnn-dropout_epoch="+str(epoch)+"_early-model.pt"
class ConvBlock2D(nn.Module):
  def __init__(self,in_channel,kernel_size,stride=1,padding=1,out_channel=None,pool=True):
    super(ConvBlock2D, self).__init__()
    if out_channel is None:
      out_channel = in_channel
    self.pool = pool

    self.layers = nn.Sequential(
      nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
      nn.BatchNorm2d(out_channel),
      nn.ReLU(),
    )

    if self.pool:
      self.pool_layer = nn.MaxPool2d(3, stride=2,padding=1) # shrinks width

  def forward(self, x):
    out = self.layers(x)

    if self.pool:
      out = self.pool_layer(out)

    return out

# Convolution - BILSTM
class Model(nn.Module):
    def __init__(self, in_size, out_vocab, hidden_size):
        super(Model, self).__init__()


        self.conv_layers = nn.Sequential(
            ConvBlock2D(1,(9,9),stride=1,padding=4,out_channel=64),                     # width is shrunk
            ConvBlock2D(64,(3,3),stride=1,padding=1,out_channel=64,pool=False),
            nn.Dropout(p=0.5),
        )

        # can add linear layer to reduce dim



        self.lstm = nn.LSTM(40*32, hidden_size*4, bidirectional=True, num_layers=2)

        self.dnn_layers = nn.Sequential(
            nn.Linear(hidden_size*8, hidden_size*4),
            nn.Linear(hidden_size*4, hidden_size*2),
            nn.Linear(hidden_size*2, out_vocab),
        )

    
    def forward(self, X, lengths):

        lengths = (lengths +1)//2

        print("X shape 1",X.shape)
        X = X.transpose(0,1).transpose(1,2).unsqueeze(1)
        print("X shape 2",X.shape)
        X = self.conv_layers(X)
        print("X shape 2.5",X.shape)
        X = torch.flatten(X, start_dim=1, end_dim=2)
        print("X shape 3",X.shape)
        X = X.squeeze(1).transpose(1,2).transpose(0,1)
        print("X shape 4",X.shape)
        

        packed_X = pack_padded_sequence(X, lengths,enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        
        out, out_lens = pad_packed_sequence(packed_out)

        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.dnn_layers(out).log_softmax(2)
        
        return out, out_lens

In [ ]:
# Old model from "/content/gdrive/MyDrive/11785hw3p2/cldnn-without_dropout-small-DNN_epoch="+str(epoch)+"_find_key.pt"

class ConvBlock2D(nn.Module):
  def __init__(self,in_channel,kernel_size,stride=1,padding=1,out_channel=None,pool=True):
    super(ConvBlock2D, self).__init__()
    if out_channel is None:
      out_channel = in_channel
    self.pool = pool

    self.layers = nn.Sequential(
      nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
      nn.BatchNorm2d(out_channel),
      nn.ReLU(),
    )

    if self.pool:
      self.pool_layer = nn.MaxPool2d(3, stride=2,padding=1) # shrinks width

  def forward(self, x):
    out = self.layers(x)

    if self.pool:
      out = self.pool_layer(out)

    return out

# Convolution - BILSTM
class Model(nn.Module):
    def __init__(self, in_size, out_vocab, hidden_size):
        super(Model, self).__init__()


        self.conv_layers = nn.Sequential(
            ConvBlock2D(1,(9,9),stride=1,padding=4,out_channel=64),                     # width is shrunk
            ConvBlock2D(64,(3,3),stride=1,padding=1,out_channel=64,pool=False),
            #nn.Dropout(p=0.5),
        )

        # can add linear layer to reduce dim



        self.lstm = nn.LSTM(40*32, hidden_size*4, bidirectional=True, num_layers=2)

        self.dnn_layers = nn.Sequential(
            #nn.Linear(hidden_size*8, hidden_size*4),
            #nn.Linear(hidden_size*4, hidden_size*2),
            #nn.Linear(hidden_size*2, out_vocab),
            nn.Linear(hidden_size*8, out_vocab),
        )

    
    def forward(self, X, lengths):

        lengths = (lengths +1)//2

        #print("X shape 1",X.shape)
        X = X.transpose(0,1).transpose(1,2).unsqueeze(1)
        #print("X shape 2",X.shape)
        X = self.conv_layers(X)
        #print("X shape 2.5",X.shape)
        X = torch.flatten(X, start_dim=1, end_dim=2)
        #print("X shape 3",X.shape)
        X = X.squeeze(1).transpose(1,2).transpose(0,1)
        #print("X shape 4",X.shape)
        

        packed_X = pack_padded_sequence(X, lengths,enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        
        out, out_lens = pad_packed_sequence(packed_out)

        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.dnn_layers(out).log_softmax(2)
        
        return out, out_lens